The key features of our application depend on python **requests** and **Beautiful Soup**

In [107]:
import csv
from datetime import datetime
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup

to start we use the indeed url template we can add search fields to

In [ ]:
template = 'https://www.indeed.com/jobs?q={}&l={}'


def get_url(position, location):
    # """Genearte a url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

We then make an http request, save the html response, and divide the response into individual cards

In [59]:
url = get_url('software engineer', 'boston ma')

# Next is to send a git request to the website
# And extact raw html

# This will send a request to the site and the response will be sent back
response = requests.get(url)
# If request was sucessful the response will return a code of 200
response
# <Response [200]>
response.reason
# 'OK'
soup = BeautifulSoup(response.text, 'html.parser')
# starting with soup obj, we use the findAll method to find all div tags that have jobsearch-SerpJobCard
cards = soup.findAll('div', 'jobsearch-SerpJobCard')
# Shows how many jobs are listed on the page
print(len(cards))
# 15

15


Using the beatiful soup parser, we search each card for data regarding the job posting

In [67]:
card = cards[1]
atag = card.h2.a
    # Use the get method because if the property doesnt exist it will return none
    # index mehtod will stop with an atribute error and thats what we don't want
    # job_title = atag.get['title']
job_url = 'https://www.indeed.com' + atag.get('href')

    # FIND COMPANY
company = card.find('span', 'company').text.strip()
    # <span class="company">

    # JOB LOCATION
    # repJobLoc is a little cleaner in the html so we will use that
try:
    job_location = card.find('span', attrs={'class': 'location'}).text
except:
    try: 
        job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except AttributeError:
        job_location = 'No location found'

    # JOB SUMMARY
try:
    job_summary = card.find('div', 'summary').text.strip()
except AttributeError:
    job_summary = 'No job summary listed'

    # POST DATE
try:
    post_date = card.find('span', 'date').text
except AttributeError:
    post_date = 'No post date found'
today = datetime.today().strftime('%Y-%m-%d')

    # SALARY
    # if the job salary is not displayed we will use a try catch
try:
    job_salary = card.find('span', 'salaryText').text.strip()
except AttributeError:
    job_salary = 'No job salary listed'

row = [company, job_salary, job_summary, post_date, job_location]
print(row)

['Raptor Maps', 'No job salary listed', 'We’re looking for an incredibly talented full-time software engineer.\nYou have 1+ years of work experience in software development.', '30+ days ago', 'Somerville, MA 02143 (Union Square area)']


We can repeat this process for each card on the page

In [71]:
data = []
for card in cards:
    atag = card.h2.a
    # Use the get method because if the property doesnt exist it will return none
    # index mehtod will stop with an atribute error and thats what we don't want
    # job_title = atag.get['title']
    job_url = 'https://www.indeed.com' + atag.get('href')

    # FIND COMPANY
    company = card.find('span', 'company').text.strip()
    # <span class="company">

    # JOB LOCATION
    # repJobLoc is a little cleaner in the html so we will use that
    try:
        job_location = card.find('span', attrs={'class': 'location'}).text
    except:
        try: 
            job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
        except AttributeError:
        job_location = 'No location found'

    # JOB SUMMARY
    try:
        job_summary = card.find('div', 'summary').text.strip()
    except AttributeError:
        job_summary = 'No job summary listed'

    # POST DATE
    try:
        post_date = card.find('span', 'date').text
    except AttributeError:
        post_date = 'No post date found'
    today = datetime.today().strftime('%Y-%m-%d')

    # SALARY
    # if the job salary is not displayed we will use a try catch
    try:
        job_salary = card.find('span', 'salaryText').text.strip()
    except AttributeError:
        job_salary = 'No job salary listed'
    row = [company, job_salary, job_summary, post_date, job_location]
    data.append(row)
    

IndentationError: expected an indented block (<ipython-input-71-d46d4871f6d3>, line 21)

Lastly, we can add the data we've collected to a dataframe for visualization and manipulation

In [72]:
titles = ['Company', 'Salary', 'Summary', 'Posting Date', 'Location']
df = pd.DataFrame(data, columns=titles)

In [73]:
df.set_index(df.Company, inplace = True)
df.drop('Company',
  axis='columns', inplace=True)
df

,Salary,Summary,Posting Date,Location
Company,,,,
Harvard University,No job salary listed,No job summary listed,No post date found,No location found
Raptor Maps,No job salary listed,We’re looking for an incredibly talented full-...,30+ days ago,"Somerville, MA 02143 (Union Square area)"
"Datawire, Inc.","$85,165 - $169,373 a year",Plus: Experience working on a cloud software a...,18 days ago,"Boston, MA 02111 (Leather District area)"
One Door,No job salary listed,No job summary listed,No post date found,No location found
PayPal,No job salary listed,No job summary listed,No post date found,No location found
Twitter,No job salary listed,No job summary listed,No post date found,No location found
Massachusetts General Hospital(MGH),No job salary listed,No job summary listed,No post date found,No location found
Revature,No job salary listed,No job summary listed,No post date found,No location found
Mavrck,No job salary listed,Execute full software development life cycle.\...,18 days ago,"Boston, MA 02114 (North End area)"


Next we want to parse data from **each page** available from our initial search field

In [99]:
pagination = soup.find('div', 'pagination')
#taken from https://stackoverflow.com/questions/43814754/python-beautifulsoup-how-to-get-href-attribute-of-a-element/43815538
links = [a['href'] for a in pagination.find_all('a', href=True)]
print(links)

['/jobs?q=software+engineer&l=boston+ma&start=10', '/jobs?q=software+engineer&l=boston+ma&start=20', '/jobs?q=software+engineer&l=boston+ma&start=30', '/jobs?q=software+engineer&l=boston+ma&start=40', '/jobs?q=software+engineer&l=boston+ma&start=10']


In [101]:
urls = ['https://www.indeed.com' + str(link) for link in links]
print(urls)

['https://www.indeed.com/jobs?q=software+engineer&l=boston+ma&start=10', 'https://www.indeed.com/jobs?q=software+engineer&l=boston+ma&start=20', 'https://www.indeed.com/jobs?q=software+engineer&l=boston+ma&start=30', 'https://www.indeed.com/jobs?q=software+engineer&l=boston+ma&start=40', 'https://www.indeed.com/jobs?q=software+engineer&l=boston+ma&start=10']


We can then process each page through a loop, ensuring that we delay our requests to conform with indeeds robots.txt

In [109]:
data = []
for url in urls:  
    response = requests.get(url)
    # If request was sucessful the response will return a code of 200
    response
    # <Response [200]>
    response.reason
    # 'OK'
    soup = BeautifulSoup(response.text, 'html.parser')
    # starting with soup obj, we use the findAll method to find all div tags that have jobsearch-SerpJobCard
    cards = soup.findAll('div', 'jobsearch-SerpJobCard')
    # Shows how many jobs are listed on the page
    print(len(cards))
    # 15
    for card in cards:
        atag = card.h2.a
        # Use the get method because if the property doesnt exist it will return none
        # index mehtod will stop with an atribute error and thats what we don't want
        # job_title = atag.get['title']
        job_url = 'https://www.indeed.com' + atag.get('href')

        # FIND COMPANY
        company = card.find('span', 'company').text.strip()
        # <span class="company">

        # JOB LOCATION
        # repJobLoc is a little cleaner in the html so we will use that
        try:
            job_location = card.find('span', attrs={'class': 'location'}).text
        except:
            try: 
                job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
            except AttributeError:
                job_location = 'No location found'

        # JOB SUMMARY
        try:
            job_summary = card.find('div', 'summary').text.strip()
        except AttributeError:
            job_summary = 'No job summary listed'

        # POST DATE
        try:
            post_date = card.find('span', 'date').text
        except AttributeError:
            post_date = 'No post date found'
        today = datetime.today().strftime('%Y-%m-%d')

        # SALARY
        # if the job salary is not displayed we will use a try catch
        try:
            job_salary = card.find('span', 'salaryText').text.strip()
        except AttributeError:
            job_salary = 'No job salary listed'
        row = [company, job_salary, job_summary, post_date, job_location]
        data.append(row)
    sleep(1)

15
15
15
15
15


In [106]:
titles = ['Company', 'Salary', 'Summary', 'Posting Date', 'Location']
df = pd.DataFrame(data, columns=titles)
df.set_index(df.Company, inplace = True)
df.drop('Company', axis='columns', inplace=True)
df

,Salary,Summary,Posting Date,Location
Company,,,,
Revature,No job salary listed,No job summary listed,No post date found,No location found
Nexthink,No job salary listed,Put in place the necessary front-end test auto...,30+ days ago,"Boston, MA"
Facebook,No job salary listed,No job summary listed,No post date found,No location found
Sun Life Financial,No job salary listed,No job summary listed,No post date found,No location found
Facebook,No job salary listed,No job summary listed,No post date found,No location found
Black Spectacles,No job salary listed,You have experience coaching junior to mid-lev...,16 days ago,"Boston, MA"
Mavrck,No job salary listed,Execute full software development life cycle.\...,18 days ago,"Boston, MA 02114 (North End area)"
Boston Children's Hospital,No job salary listed,No job summary listed,No post date found,No location found
Raytheon Intelligence & Space,No job salary listed,No job summary listed,No post date found,No location found


In [110]:
df[df['Summary'] != 'No job summary listed']

,Salary,Summary,Posting Date,Location
Company,,,,
Nexthink,No job salary listed,Put in place the necessary front-end test auto...,30+ days ago,"Boston, MA"
Black Spectacles,No job salary listed,You have experience coaching junior to mid-lev...,16 days ago,"Boston, MA"
Mavrck,No job salary listed,Execute full software development life cycle.\...,18 days ago,"Boston, MA 02114 (North End area)"
Recorded Future,No job salary listed,Python programming: You are comfortable writin...,11 days ago,"Somerville, MA"
HqO,No job salary listed,Evolve our technology in alignment with our ar...,30+ days ago,"Boston, MA 02111 (Central area)"
ZeroNorth Inc,No job salary listed,"Experience working with cloud platforms (AWS, ...",30+ days ago,"Boston, MA 02111 (Central area)"
"Numerated Growth Technologies, Inc.",No job salary listed,"As a front end focused Software Engineer, you’...",30+ days ago,"Boston, MA 02110 (Central area)"
Dark Wolf Solutions,No job salary listed,Our software engineers are asked to solve chal...,4 days ago,"Boston, MA"
Wasabi Technologies,No job salary listed,"You will be working in a Linux environment, us...",30+ days ago,"Boston, MA 02199 (Back Bay area)"
